In [1]:
point_A = [55.75222, 37.61556]  # Moscow
point_B = [50.45466, 30.5238]   # Kyiv

num_waypoints = 5
num_paths = 3

In [2]:
from terrain import calculate_distance

straight_line_distance:float = calculate_distance(
    point_A[0],
    point_A[1],
    point_B[0],
    point_B[1]
)

print(f'Straight line distance: {straight_line_distance} km')

Straight line distance: 756.2941363684055 km


In [3]:
import random
from geopy.distance import great_circle

def generate_random_waypoints(start:tuple[float, float], end:tuple[float, float], num_waypoints:int) -> list[tuple[int,int]]:
    """
    Generates random waypoints between a start and endpoint.
    
    Args: 
        start (tuple[float, float]): starting latitude and longitude.
        end (tuple[float, float]): ending latitude and longitude.
        num_waypoints (int) number of random waypoints to generate.
    
    Returns 
        list[tuple[float, float]]: list of tuples containing the waypoints.
    """
    waypoints:list[tuple[float, float]] = [start]
    
    # Generate random waypoints
    for _ in range(num_waypoints):
        # Randomly choose a fraction between 0 and 1 
        fraction = random.uniform(0, 1)

        # Add the fraction to the lat/long of the start and end to generate a random point between the two
        lat:float = start[0] + fraction * (end[0] - start[0])
        lon:float = start[1] + fraction * (end[1] - start[1])
        waypoints.append((lat, lon))
    
    # Add the end location to the waypoints so we end in the right place 
    waypoints.append(end)

    # Return the populated list 
    return waypoints


def create_random_paths(start:tuple[float, float], end:tuple[float, float], num_waypoints:int, num_paths:int) -> list[list[tuple[float, float]]]:
    """
    Creates multiple random paths between a start and endpoint.
    
    Args: 
        start (tuple[float, float]): starting latitude and longitude.
        end (tuple[float, float]): ending latitude and longitude.
        num_waypoints (int): number of random waypoints to generate.
        num_paths (int): number of paths to generate

    Returns: 
        list[list[tuple[float, float]]]: list of lists containing paths, where each path is a list of tuples (lat, lon).
    """
    # Init empty list of paths
    paths = []
    
    # Generate num_paths possible paths 
    for _ in range(num_paths):
        waypoints = generate_random_waypoints(start, end, num_waypoints)
        paths.append(waypoints)
    
    return paths

In [4]:
random_paths = create_random_paths(point_A, point_B, num_waypoints, num_paths)

# Print generated paths
for i, path in enumerate(random_paths):
    print(f"Path {i + 1}:")
    for waypoint in path:
        print(f"  {waypoint}")
    print()

Path 1:
  [55.75222, 37.61556]
  (53.07148412943991, 34.02690118020311)
  (54.74331596869371, 36.26495608932099)
  (55.581286885253824, 37.38673453585569)
  (52.900626601539095, 33.79817690297626)
  (53.093317026406055, 34.05612853494541)
  [50.45466, 30.5238]

Path 2:
  [55.75222, 37.61556]
  (51.594615659367825, 32.04984065775157)
  (53.45130745524334, 34.53536467452874)
  (55.05776724074017, 36.68590704656323)
  (51.82386692899023, 32.356735715826865)
  (52.65942367567921, 33.47528235123996)
  [50.45466, 30.5238]

Path 3:
  [55.75222, 37.61556]
  (54.94617308142994, 36.536517705955504)
  (54.7345910911126, 36.25327623334303)
  (53.47931368353897, 34.572856170534045)
  (53.27918295407254, 34.30494432017258)
  (51.806780616895466, 32.33386253937183)
  [50.45466, 30.5238]

